In [16]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [ ]:
#데이터 분할을 위한 폴더 생성
import os
import shutil

original_dataset_dir='C:/Users/user/Desktop/kang/dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir='C:/Users/user/Desktop/kang/spliteed'
os.mkdir(base_dir)

train_dir=os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir=os.path.join(base_dir,'val')
os.mkdir(validation_dir)
test_dir=os.path.join(base_dir,'test')
os.mkdir(test_dir)

for clss in classes_list:
    os.mkdir(os.path.join(train_dir,clss))
    os.mkdir(os.path.join(validation_dir,clss))
    os.mkdir(os.path.join(test_dir,clss))

In [17]:
#데이터 분할과 클래스별 데이터 수 확인
import math

for clss in classes_list:
    path=os.path.join(original_dataset_dir,clss)
    fnames=os.listdir(path)
    
    train_size=math.floor(len(fnames) * 0.6)
    validation_size=math.floor(len(fnames) * 0.2)
    test_size=math.floor(len(fnames)* 0.2)
    
    train_fnames=fnames[:train_size]
    print('Train size(',clss,'): ',len(train_fnames))
    for fname in train_fnames:
        src=os.path.join(path, fname)
        dst=os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src,dst)
    
    validation_fnames=fnames[train_size:(validation_size + train_size)]
    print('Validation size(',clss,'): ',len(validation_fnames))
    for fname in validation_fnames:
        src=os.path.join(path, fname)
        dst=os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src,dst)
    
    test_fnames=fnames[(train_size + validation_size):(validation_size + train_size + test_size)]
    print('Test size(',clss,'): ', len(test_fnames))
    for fname in test_fnames:
        src=os.path.join(path,fname)
        dst=os.path.join(os.path.join(test_dir,clss),fname)
        shutil.copyfile(src,dst)

Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124
Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329
Train size( Cherry___healthy ):  512
Validation size( Cherry___healthy ):  170
Test size( Cherry___healthy ):  170
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  307
Validation size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Test size( Corn___Cercospora_leaf_spot Gray_leaf_spot ):  102
Train size( Corn___Common_rust ):  715
Validation size( Corn___Commo

In [19]:
#베이스라인 모델 학습을 위한 준비
import torch

USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)

BATCH_SIZE =256
EPOCH=30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base=transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

train_dataset = ImageFolder(root='C:/Users/user/Desktop/kang/spliteed/train', transform=transform_base)
val_dataset = ImageFolder(root='C:/Users/user/Desktop/kang/spliteed/val', transform=transform_base)

from torch.utils.data import DataLoader

train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader=torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

cuda


In [25]:
#베이스라인 모델 설계
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.conv1=nn.Conv2d(3,32,3,padding=1)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(32,64,3,padding=1)
        self.conv3=nn.Conv2d(64,64,3,padding=1)
        
        self.fc1=nn.Linear(4096,512)
        self.fc2=nn.Linear(512,33)
        
    def forward(self,x):
        
        x=self.conv1(x)
        x=F.relu(x)
        x=self.pool(x)
        x=F.dropout(x, p=0.25, training=self.training)
        
        x=self.conv2(x)
        x=F.relu(x)
        x=self.pool(x)
        x=F.dropout(x, p=0.25, training=self.training)
        
        x=self.conv3(x)
        x=F.relu(x)
        x=self.pool(x)
        x=F.dropout(x, p=0.25, training=self.training)
        
        x=x.view(-1,4096)
        x=self.fc1(x)
        x=F.relu(x)
        x=F.dropout(x, p=0.5, training=self.training)
        x=self.fc2(x)
        
        return F.log_softmax(x,dim=1)
    
model_base=Net().to(DEVICE)
optimizer=optim.Adam(model_base.parameters(), lr=0.001)

In [26]:
#모델 학습을 위한 함수
def train(model, train_loader, optimizer):
    model.train()
    for batch_indx, (data,target) in enumerate(train_loader):
        data, target=data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()

In [27]:
#모델 평가를 위한 함수
def evaluate(model, test_loader):
    model.eval()
    test_loss=0
    correct=0
    
    with torch.no_grad():
        for data, target in test_loader:
            data,target=data.to(DEVICE), target.to(DEVICE)
            output=model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred=output.max(1,keepdim=True)[1]
            correct+=pred.eq(target.view_as(pred)).sum().item()
    test_loss /=len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [28]:
#모델 학습 실행하기
import time
import copy

def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30):
    best_acc=0.0
    best_model_wts=copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs+1):
        since=time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc=evaluate(model, train_loader)
        val_loss, val_acc=evaluate(model, val_loader)
        
        if val_acc > best_acc:
            best_acc=val_acc
            best_model_wts=copy.deepcopy(model.state_dict())
        time_elapsed=time.time()-since
        print(f"epoch {epoch}")
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss,train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss,val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed //60, time_elapsed %60))
    model.load_state_dict(best_model_wts)
    return model
base=train_baseline(model_base, train_loader,val_loader,optimizer,EPOCH)
torch.save(base,'baseline.pt')

epoch 1
train Loss: 1.9976, Accuracy: 43.34%
val Loss: 2.0098, Accuracy: 42.93%
Completed in 1m 17s
epoch 2
train Loss: 1.1228, Accuracy: 66.16%
val Loss: 1.1623, Accuracy: 65.15%
Completed in 0m 49s
epoch 3
train Loss: 0.8093, Accuracy: 75.47%
val Loss: 0.8603, Accuracy: 73.90%
Completed in 0m 50s
epoch 4
train Loss: 0.6350, Accuracy: 80.43%
val Loss: 0.7026, Accuracy: 78.56%
Completed in 0m 49s
epoch 5
train Loss: 0.6286, Accuracy: 79.74%
val Loss: 0.6989, Accuracy: 77.02%
Completed in 0m 49s
epoch 6
train Loss: 0.5099, Accuracy: 83.37%
val Loss: 0.5865, Accuracy: 80.59%
Completed in 0m 49s
epoch 7
train Loss: 0.4041, Accuracy: 87.20%
val Loss: 0.4912, Accuracy: 84.23%
Completed in 0m 49s
epoch 8
train Loss: 0.3733, Accuracy: 88.49%
val Loss: 0.4634, Accuracy: 85.58%
Completed in 0m 48s
epoch 9
train Loss: 0.3025, Accuracy: 91.16%
val Loss: 0.3935, Accuracy: 87.70%
Completed in 0m 49s
epoch 10
train Loss: 0.2985, Accuracy: 90.60%
val Loss: 0.4013, Accuracy: 86.78%
Completed in 0m 48s